# Ray RLlib - Extra Application Example - Taxi-v3

© 2019-2021, Anyscale. All Rights Reserved

This example uses [RLlib](https://ray.readthedocs.io/en/latest/rllib.html) to train a policy with the `Taxi-v3` environment ([gym.openai.com/envs/Taxi-v3/](https://gym.openai.com/envs/Taxi-v3/)). The goal is to pick up passengers as fast as possible, negotiating the available paths. This is one of OpenAI Gym's ["toy text"](https://gym.openai.com/envs/#toy_text) problems.

For more background about this problem, see:

* ["Hierarchical Reinforcement Learning with the MAXQ Value Function Decomposition"](https://arxiv.org/abs/cs/9905014), [Thomas G. Dietteric](https://twitter.com/tdietterich)
* ["Reinforcement Learning: let’s teach a taxi-cab how to drive"](https://towardsdatascience.com/reinforcement-learning-lets-teach-a-taxi-cab-how-to-drive-4fd1a0d00529), [Valentina Alto](https://twitter.com/AltoValentina)

In [1]:
import pandas as pd
import json
import os
import shutil
import sys
import ray
import ray.rllib.agents.ppo as ppo

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
info = ray.init(ignore_reinit_error=True)

2021-06-03 16:17:40,870	INFO services.py:1267 -- View the Ray dashboard at http://127.0.0.1:8265


In [3]:
print("Dashboard URL: http://{}".format(info["webui_url"]))

Dashboard URL: http://127.0.0.1:8265


Set up the checkpoint location:

In [4]:
checkpoint_root = "tmp/ppo/taxi"
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)   # clean up old runs

Next we'll train an RLlib policy with the `Taxi-v3` environment.

By default, training runs for `10` iterations. Increase the `N_ITER` setting if you want to see the resulting rewards improve.
Also note that *checkpoints* get saved after each iteration into the `/tmp/ppo/taxi` directory.

> **Note:** If you prefer to use a different directory root than `/tmp`, change it in the next cell **and** in the `rllib rollout` command below.

In [5]:
SELECT_ENV = "Taxi-v3"
N_ITER = 10

config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"

agent = ppo.PPOTrainer(config, env=SELECT_ENV)

2021-06-03 16:17:43,079	INFO trainer.py:669 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-06-03 16:17:43,081	INFO trainer.py:694 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=164530) WARNING:tensorflow:From /home/rhys/Projects/RLxEvolution/RLxEvolution/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=164530) Instructions for updating:
(pid=164530) non-resource variables are not supported in the long term
(pid=164525) WARNING:tensorflow:From /home/rhys/Projects/RLxEvolution/RLxEvolution/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=164525) Instructions for updating:
(pid=

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2021-06-03 16:17:47,323	WARNING util.py:53 -- Install gputil for GPU system monitoring.


In [6]:
results = []
episode_data = []
episode_json = []

for n in range(N_ITER):
    result = agent.train()
    results.append(result)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(checkpoint_root)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

2021-06-03 16:17:49,880	WARNING deprecation.py:33 -- DeprecationWarning: `SampleBatch.data[..]` has been deprecated. Use `SampleBatch[..]` instead. This will raise an error in the future!


  1: Min/Mean/Max reward: -884.0000/-798.0500/-686.0000, len mean: 200.0000. Checkpoint saved to tmp/ppo/taxi/checkpoint_000001/checkpoint-1
  2: Min/Mean/Max reward: -884.0000/-769.2500/-650.0000, len mean: 200.0000. Checkpoint saved to tmp/ppo/taxi/checkpoint_000002/checkpoint-2
  3: Min/Mean/Max reward: -884.0000/-744.5000/-542.0000, len mean: 200.0000. Checkpoint saved to tmp/ppo/taxi/checkpoint_000003/checkpoint-3
  4: Min/Mean/Max reward: -884.0000/-728.8625/-542.0000, len mean: 200.0000. Checkpoint saved to tmp/ppo/taxi/checkpoint_000004/checkpoint-4
  5: Min/Mean/Max reward: -884.0000/-712.5500/-515.0000, len mean: 200.0000. Checkpoint saved to tmp/ppo/taxi/checkpoint_000005/checkpoint-5
  6: Min/Mean/Max reward: -848.0000/-675.1100/-434.0000, len mean: 200.0000. Checkpoint saved to tmp/ppo/taxi/checkpoint_000006/checkpoint-6
  7: Min/Mean/Max reward: -848.0000/-643.2500/-425.0000, len mean: 200.0000. Checkpoint saved to tmp/ppo/taxi/checkpoint_000007/checkpoint-7
  8: Min/Mean

Do the episode rewards increase after multiple iterations?

Also, print out the policy and model to see the results of training in detail…

In [7]:
import pprint

policy = agent.get_policy()
model = policy.model

pprint.pprint(model.variables())
pprint.pprint(model.value_function())

print(model.base_model.summary())

[<tf.Variable 'default_policy/fc_1/kernel:0' shape=(500, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_1/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_1/kernel:0' shape=(500, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_1/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_2/kernel:0' shape=(256, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_2/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_2/kernel:0' shape=(256, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_2/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_out/kernel:0' shape=(256, 6) dtype=float32>,
 <tf.Variable 'default_policy/fc_out/bias:0' shape=(6,) dtype=float32>,
 <tf.Variable 'default_policy/value_out/kernel:0' shape=(256, 1) dtype=float32>,
 <tf.Variable 'default_policy/value_out/bias:0' shape=(1,) dtype=float32>]
<tf.Tensor 'Reshape:0' shape=(?,) dtype=float32>
Model: "model"
______

## Rollout

Next we'll use the [`rollout` script](https://ray.readthedocs.io/en/latest/rllib-training.html#evaluating-trained-policies) to evaluate the trained policy.

The output from the following command visualizes the "taxi" agent operating within its simulation: picking up a passenger, driving, turning, dropping off a passenger ("put-down"), and so on. 

A 2-D map of the *observation space* is visualized as text, which needs some decoding instructions:

  * `R` -- R(ed) location in the Northwest corner
  * `G` -- G(reen) location in the Northeast corner
  * `Y` -- Y(ellow) location in the Southwest corner
  * `B` -- B(lue) location in the Southeast corner
  * `:` -- cells where the taxi can drive
  * `|` -- obstructions ("walls") which the taxi must avoid
  * blue letter represents the current passenger’s location for pick-up
  * purple letter represents the drop-off location
  * yellow rectangle is the current location of our taxi/agent

That allows for a total of 500 states, and these known states are numbered between 0 and 499.

The *action space* for the taxi/agent is defined as:

  * move the taxi one square North
  * move the taxi one square South
  * move the taxi one square East
  * move the taxi one square West
  * pick-up the passenger
  * put-down the passenger

The *rewards* are structured as −1 for each action plus:

 * +20 points when the taxi performs a correct drop-off for the passenger
 * -10 points when the taxi attempts illegal pick-up/drop-off actions

Admittedly it'd be better if these state visualizations showed the *reward* along with observations.

In [8]:
!rllib rollout \
    tmp/ppo/taxi/checkpoint_000010/checkpoint-10 \
    --config "{\"env\": \"Taxi-v3\"}" \
    --run PPO \
    --steps 2000

Instructions for updating:
non-resource variables are not supported in the long term
Traceback (most recent call last):
  File "/home/rhys/Projects/RLxEvolution/RLxEvolution/bin/rllib", line 8, in <module>
    sys.exit(cli())
  File "/home/rhys/Projects/RLxEvolution/RLxEvolution/lib/python3.8/site-packages/ray/rllib/scripts.py", line 36, in cli
    rollout.run(options, rollout_parser)
  File "/home/rhys/Projects/RLxEvolution/RLxEvolution/lib/python3.8/site-packages/ray/rllib/rollout.py", line 277, in run
    raise ValueError(
ValueError: Could not find params.pkl in either the checkpoint dir or its parent directory AND no `--config` given on command line!


In [9]:
ray.shutdown()  # "Undo ray.init()".

## Exercise ("Homework")

In addition to _Taxi_, there are other so-called ["toy text"](https://gym.openai.com/envs/#toy_text) problems you can try.